In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [2]:
import wbdata #used to fetch economic and developmeent data from the world bank API
import pandas as pd
import numpy as np

import plotly.express as px
import datetime 
import time


from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import ipywidgets as widgets
from IPython.display import display 


In [3]:
eu_countries = [            # List of ISO-2 codes representing all 27 EU member countries
    'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI',
    'FR', 'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU',
    'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'
] 

# Dictionary mapping world  bank indicator codes to human - readable labels
indicators = {
    'NY.GDP.PCAP.CD': 'GDP per capita',                            # GDP per person (US dollars)
    'SE.XPD.TOTL.GD.ZS': 'Education spending (% GDP)',            # % of GDP spent on education
    'SH.XPD.CHEX.PC.CD': 'Health spending per capita (US$)',      # Health spending per person
    'SP.POP.TOTL': 'Population',                                   # Total population
    'NE.GDI.TOTL.ZS': 'Investment (% GDP)',                        # Gross capital formation
    'SL.TLF.CACT.ZS': 'Labor force participation (%)' 
}

In [4]:
# Fetching data from the World Bank API 
df_raw = wbdata.get_dataframe(indicators, country=eu_countries)

# Reseting the multi-index to convert 'country' and 'date' into columns
df_raw.reset_index(inplace=True)

In [5]:
# data cleaning and preprocessing 
# converts the 'date' column to datetime format
df_raw['date'] = pd.to_datetime(df_raw['date'])

# Extracting just the year from the datetime object into a new 'year' column
df_raw['year'] = df_raw['date'].dt.year

# filtering all the rows  before the year 2000 and droping na values
df_raw = df_raw[df_raw['year'] >= 2000].dropna()

# Mapping countries names to ISO-2 codes (for consistency or future mapping)

df_raw['country_code'] = df_raw['country'].map({
    'Austria': 'AT', 'Belgium': 'BE', 'Bulgaria': 'BG', 'Croatia': 'HR',
    'Cyprus': 'CY', 'Czech Republic': 'CZ', 'Denmark': 'DK', 'Estonia': 'EE',
    'Finland': 'FI', 'France': 'FR', 'Germany': 'DE', 'Greece': 'GR',
    'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Latvia': 'LV',
    'Lithuania': 'LT', 'Luxembourg': 'LU', 'Malta': 'MT', 'Netherlands': 'NL',
    'Poland': 'PL', 'Portugal': 'PT', 'Romania': 'RO', 'Slovakia': 'SK',
    'Slovenia': 'SI', 'Spain': 'ES', 'Sweden': 'SE'
})

print('EU dataset shape:', df_raw.shape)
df_raw.head(5)

EU dataset shape: (600, 10)


,country,date,GDP per capita,Education spending (% GDP),Health spending per capita (US$),Population,Investment (% GDP),Labor force participation (%),year,country_code
2,Austria,2022-01-01,52176.664914,4.774471,5851.962402,9041851.0,29.226770,60.882,2022,AT
3,Austria,2021-01-01,53648.719074,5.494110,6520.490723,8955797.0,27.673244,60.631,2021,AT
4,Austria,2020-01-01,48716.409890,5.571270,5531.023438,8916864.0,25.653074,60.364,2020,AT
5,Austria,2019-01-01,49885.994736,5.217790,5263.390625,8879920.0,25.509008,60.898,2019,AT
6,Austria,2018-01-01,51194.074984,5.226550,5337.218262,8840521.0,25.898276,60.919,2018,AT


## Country Clustering Dashboard – KMeans Clustering on Socioeconomic Indicators

### Variables Used:
- GDP per capita
- Health spending per capita (US$)
- Education spending (% GDP)
- Investment (% GDP)
- Labor force participation (%)
- Population

### Computing average values per country

In [6]:
#Grouping data by country and calculating the mean for each selected variable
#droping any countries with missing values
average_df = df_raw.groupby('country')[[
    'GDP per capita',
    'Health spending per capita (US$)',
    'Education spending (% GDP)',
    'Investment (% GDP)',
    'Labor force participation (%)',
    'Population'
]].mean().dropna()

### Normalizing the data 

In [7]:
# standardizing all columns in the dataset so that they have a mean of 0 and standard deviation of = 1
scaler  = StandardScaler()
scaled_data = scaler.fit_transform(average_df)

In [8]:
# applying K-Means Clustering 
number_clusters = 4
kmeans= KMeans(n_clusters=number_clusters, random_state=0, n_init=10)

#fitting KMeans to the scaled data and assing each country to a cluster 
average_df['Cluster'] = kmeans.fit_predict(scaled_data)

In [ ]:
#Reseting index so 'country' becomes acolumn again for plotting 
